# Phishing Website Detection - Feature Extraction


Step -1 : Data preprocessing 

This dataset contains few website links (Some of them are legitimate websites and a few are fake websites)

Pre-Processing the data before building a model and also Extracting the features from the data based on certain conditions

In [1]:
#importing numpy and pandas which are required for data pre-processing
import numpy as np
import pandas as pd

In [11]:
#Loading the data
raw_data = pd.read_csv("../raw_datasets/100-legitimate-art.txt") #loading only 100 samples (art websites data)

In [12]:
raw_data.head()

,websites
0,http://www.emuck.com:3000/archive/egan.html
1,http://danoday.com/summit.shtml
2,http://groups.yahoo.com/group/voice_actor_appr...
3,http://voice-international.com/
4,http://www.livinglegendsltd.com/


We need to split the data according to parts of the URL

A typical URL could have the form http://www.example.com/index.html, which indicates a protocol (http), a hostname (www.example.com), and a file name (index.html).

Detailed explanation about URL:  1.https://doepud.co.uk/blog/anatomy-of-a-url                                                    
                                 2.https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_is_a_URL                  
                                 3.https://techwelkin.com/understanding-the-components-and-structure-of-a-url
                                        
                                            

In [13]:
raw_data['websites'].str.split("://").head() #Here we divided the protocol from the entire URL. but need it to be divided it 
                                                 #seperate column

0         [http, www.emuck.com:3000/archive/egan.html]
1                     [http, danoday.com/summit.shtml]
2    [http, groups.yahoo.com/group/voice_actor_appr...
3                     [http, voice-international.com/]
4                    [http, www.livinglegendsltd.com/]
Name: websites, dtype: object








refer to this link for splitting of data --> https://apassionatechie.wordpress.com/2018/02/24/how-do-i-split-a-string-into-several-columns-in-a-dataframe-with-pandas-python/

In [14]:
seperation_of_protocol = raw_data['websites'].str.split("://",expand = True) #expand argument in the split method will give you a new column

In [15]:
seperation_of_protocol.head()

,0,1
0,http,www.emuck.com:3000/archive/egan.html
1,http,danoday.com/summit.shtml
2,http,groups.yahoo.com/group/voice_actor_appreciatio...
3,http,voice-international.com/
4,http,www.livinglegendsltd.com/


In [16]:
type(seperation_of_protocol)

pandas.core.frame.DataFrame

In [20]:
seperation_domain_name = seperation_of_protocol[1].str.split("/", n=1, expand=True)

In [21]:
type(seperation_domain_name)

pandas.core.frame.DataFrame

In [22]:
seperation_domain_name.columns=["domain_name","address"] #renaming columns of data frame

In [23]:
seperation_domain_name.head()

,domain_name,address
0,www.emuck.com:3000,archive/egan.html
1,danoday.com,summit.shtml
2,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...
3,voice-international.com,
4,www.livinglegendsltd.com,


In [24]:
#Concatenation of data frames
splitted_data = pd.concat([seperation_of_protocol[0],seperation_domain_name],axis=1)


In [25]:
splitted_data.columns = ['protocol','domain_name','address']

In [26]:
splitted_data.head()

,protocol,domain_name,address
0,http,www.emuck.com:3000,archive/egan.html
1,http,danoday.com,summit.shtml
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...
3,http,voice-international.com,
4,http,www.livinglegendsltd.com,


Domain name column can be further sub divided into domain_names as well as sub_domain_names 

Similarly, address column can also be further sub divided into path,query_string,file..................

In [27]:
type(splitted_data)

pandas.core.frame.DataFrame

### Features Extraction


Feature-1

1.Long URL to Hide the Suspicious Part

If the length of the URL is greater than or equal 54 characters then the URL classified as phishing


0 --- indicates legitimate

1 --- indicates Phishing

2 --- indicates Suspicious

In [28]:
def long_url(l):
    """This function is defined in order to differntiate website based on the length of the URL"""
    if len(l) < 54:
        return 0
    elif len(l) >= 54 and len(l) <= 75:
        return 2
    return 1

In [29]:
#Applying the above defined function in order to divide the websites into 3 categories
splitted_data['long_url'] = raw_data['websites'].apply(long_url) 


In [30]:
#Will show the results only the websites which are legitimate according to above condition as 0 is legitimate website
splitted_data[splitted_data.long_url == 0].head()

,protocol,domain_name,address,long_url
0,http,www.emuck.com:3000,archive/egan.html,0
1,http,danoday.com,summit.shtml,0
3,http,voice-international.com,,0
4,http,www.livinglegendsltd.com,,0
5,http,voicechasers.com,forum/viewforum.php?f=8,0


Feature-2

2.URL’s having “@” Symbol

Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

IF {Url Having @ Symbol→ Phishing
    Otherwise→ Legitimate }


0 --- indicates legitimate

1 --- indicates Phishing


In [31]:
def have_at_symbol(l):
    """This function is used to check whether the URL contains @ symbol or not"""
    if "@" in l:
        return 1
    return 0
    

In [32]:
splitted_data['having_@_symbol'] = raw_data['websites'].apply(have_at_symbol)

In [33]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol
0,http,www.emuck.com:3000,archive/egan.html,0,0
1,http,danoday.com,summit.shtml,0,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0
3,http,voice-international.com,,0,0
4,http,www.livinglegendsltd.com,,0,0


Feature-3

3.Redirecting using “//”

The existence of “//” within the URL path means that the user will be redirected to another website.
An example of such URL’s is: “http://www.legitimate.com//http://www.phishing.com”. 
We examine the location where the “//” appears. 
We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. 
However, if the URL employs “HTTPS” then the “//” should appear in seventh position.

IF {ThePosition of the Last Occurrence of "//" in the URL > 7→ Phishing
    
    Otherwise→ Legitimate

0 --- indicates legitimate

1 --- indicates Phishing


In [34]:
def redirection(l):
    """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
    if "//" in l:
        return 1
    return 0

In [35]:
splitted_data['redirection_//_symbol'] = seperation_of_protocol[1].apply(redirection)

In [36]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol
0,http,www.emuck.com:3000,archive/egan.html,0,0,0
1,http,danoday.com,summit.shtml,0,0,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0
3,http,voice-international.com,,0,0,0
4,http,www.livinglegendsltd.com,,0,0,0


Feature-4

4.Adding Prefix or Suffix Separated by (-) to the Domain

The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name
so that users feel that they are dealing with a legitimate webpage. 

For example http://www.Confirme-paypal.com/.
    
IF {Domain Name Part Includes (−) Symbol → Phishing
    
    Otherwise → Legitimate
    
1 --> indicates phishing

0 --> indicates legitimate
    

In [37]:
def prefix_suffix_seperation(l):
    if '-' in l:
        return 1
    return 0

In [38]:
splitted_data['prefix_suffix_seperation'] = seperation_domain_name['domain_name'].apply(prefix_suffix_seperation)

In [39]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0
1,http,danoday.com,summit.shtml,0,0,0,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0
3,http,voice-international.com,,0,0,0,1
4,http,www.livinglegendsltd.com,,0,0,0,0


Feature - 5

5. Sub-Domain and Multi Sub-Domains

The legitimate URL link has two dots in the URL since we can ignore typing “www.”. 
If the number of dots is equal to three then the URL is classified as “Suspicious” since it has one sub-domain.
However, if the dots are greater than three it is classified as “Phishy” since it will have multiple sub-domains

0 --- indicates legitimate

1 --- indicates Phishing

2 --- indicates Suspicious


In [40]:
def sub_domains(l):
    if l.count('.') < 3:
        return 0
    elif l.count('.') == 3:
        return 2
    return 1

In [41]:
splitted_data['sub_domains'] = splitted_data['domain_name'].apply(sub_domains)

In [42]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0
1,http,danoday.com,summit.shtml,0,0,0,0,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0,0
3,http,voice-international.com,,0,0,0,1,0
4,http,www.livinglegendsltd.com,,0,0,0,0,0


Feature-6

6.Using the IP Address

If an IP address is used as an alternative of the domain name in the URL, such as “http://125.98.3.123/fake.html”,
users can be sure that someone is trying to steal their personal information. Sometimes, 
the IP address is even transformed into hexadecimal code as shown in the following link “http://0x58.0xCC.0xCA.0x62/2/paypal.ca/index.html”.

        Rule: IF{If The Domain Part has an IP Address → Phishing
                 Otherwise→ Legitimate

                 1 --> indicates phishing

                 0 --> indicates legitimate

In [43]:
import re
def having_ip_address(url):
    match=re.search('(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  #IPv4
                    '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  #IPv4 in hexadecimal
                    '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',url)     #Ipv6
    if match:
        #print match.group()
        return 1
    else:
        #print 'No matching pattern found'
        return 0


In [44]:
splitted_data['having_ip_address'] = raw_data['websites'].apply(having_ip_address)

In [45]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0
1,http,danoday.com,summit.shtml,0,0,0,0,0,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0,0,0
3,http,voice-international.com,,0,0,0,1,0,0
4,http,www.livinglegendsltd.com,,0,0,0,0,0,0


Feature-7

7.Using URL Shortening Services “TinyURL”

URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. 
This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL. 
For example, the URL “http://portal.hud.ac.uk/” can be shortened to “bit.ly/19DXSk4”.
    
Rule: IF{TinyURL → Phishing
         
         Otherwise→ Legitimate
         
                 1 --> indicates phishing

                 0 --> indicates legitimate

In [46]:
#we have imported re module in the above feature. So need not to import again
def shortening_service(url):
    match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|tr\.im|link\.zip\.net',url)
    if match:
        return 1
    else:
        return 0



<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\.'
<>:9: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\.'
<>:9: SyntaxWarning: invalid escape sequence '\.'
C:\Users\User\AppData\Local\Temp\ipykernel_1936\1647293496.py:3: SyntaxWarning: invalid escape sequence '\.'
  match=re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
C:\Users\User\AppData\Local\Temp\ipykernel_1936\1647293496.py:4: SyntaxWarning: inva

In [47]:
splitted_data['shortening_service'] = raw_data['websites'].apply(shortening_service)

In [48]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0
1,http,danoday.com,summit.shtml,0,0,0,0,0,0,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0,0,0,0
3,http,voice-international.com,,0,0,0,1,0,0,0
4,http,www.livinglegendsltd.com,,0,0,0,0,0,0,0


Feature - 8 

8.The Existence of “HTTPS” Token in the Domain Part of the URL

The phishers may add the “HTTPS” token to the domain part of a URL in order to trick users.
For example, http://https-www-paypal-it-webapps-mpp-home.soft-hair.com/.

    Rule: IF{Using HTTP Token in Domain Part of The URL→ Phishing
             
             Otherwise→ Legitimate

In [49]:
def https_token(url):
    match=re.search('https://|http://',url)
    if match.start(0)==0:
        url=url[match.end(0):]
    match=re.search('http|https',url)
    if match:
        return 1
    else:
        return 0


In [50]:
splitted_data['https_token'] = raw_data['websites'].apply(https_token)

In [51]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0,0
1,http,danoday.com,summit.shtml,0,0,0,0,0,0,0,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0,0,0,0,0
3,http,voice-international.com,,0,0,0,1,0,0,0,0
4,http,www.livinglegendsltd.com,,0,0,0,0,0,0,0,0


Feature - 9

9.Abnormal_URL

This feature can be extracted from WHOIS database. For a legitimate website, identity is typically part of its URL.

Rule: IF {The Host Name Is Not Included In URL → Phishing 
          
          Otherwise→ Legitimate

In [52]:
def abnormal_url_sub(domain,url):
    hostname=domain.name
    match=re.search(hostname,url)
    if match:
        return 0
    else:
        return 1
    

In [55]:
def abnormal_url_main(domain,url):
    dns = 0
    try:
        domain_name = whois.whois(domain)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return abnormal_url_sub(domain_name,url)
    
    

In [58]:
def abnormal_url_main(url):
    return "abnormal" if "@" in url else "normal"

In [60]:
# ?
 #x - - cross check 
splitted_data['abnormal_url'] = splitted_data['domain_name'].apply(abnormal_url_main)

Feature - 10:
    
10.Google Index

This feature examines whether a website is in Google’s index or not. When a site is indexed by Google,
it is displayed on search results (Webmaster resources, 2014). 
Usually, phishing webpages are merely accessible for a short period and as a result, 
many phishing webpages may not be found on the Google index.

Rule: IF{Webpage Indexed by Google → Legitimate
         
         Otherwise → Phishing

In [62]:
!pip install googlesearch-python

In [64]:
# Install first:
# pip install googlesearch-python

from googlesearch import search

def google_index(url):
    try:
        site = list(search(url, num_results=5))
        if site:
            return 0
        else:
            return 1
    except Exception as e:
        print(f"Error searching Google: {e}")
        return 1


In [65]:
splitted_data['google_index'] = raw_data['websites'].apply(google_index)

Error searching Google: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3Dhttp%253A%252F%252Fwww.emuck.com%253A3000%252Farchive%252Fegan.html%26num%3D7%26hl%3Den%26start%3D0%26safe%3Dactive&hl=en&q=EgRnbHEVGICEv8AGIjAOYw5PTstquSZSnJ8iqAxVJ7TjnfWgQlRKStFyQCuCQDM17T6tp03D10KuBDNgEIEyAnJSWgFD
Error searching Google: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3Dhttp%253A%252F%252Fdanoday.com%252Fsummit.shtml%26num%3D7%26hl%3Den%26start%3D0%26safe%3Dactive&hl=en&q=EgRnbHEVGICEv8AGIjAL3Uy1w5mt1GoJBJwyhX_HWhJIKxFC2OXx7hrtGK9l0ckDoCKsqTQPLYb02IAekP8yAnJSWgFD
Error searching Google: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3Dhttp%253A%252F%252Fgroups.yahoo.com%252Fgroup%252Fvoice_actor_appreciation%252Flinks%252Fevents_and_appearanc_000985383761%252F%26num%3D7%2

Feature - 11

11.Website Traffic

This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit.
However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database 
(Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios,
legitimate websites ranked among the top 100,000. Furthermore,
if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.
Otherwise, it is classified as “Suspicious”.

Rule: IF{Website Rank<100,000 → LegitimateWebsite Rank>100,
         
         000 →SuspiciousOtherwise → Phish

In [70]:
import urllib.request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

def web_traffic(url):
    try:
        if not url.startswith(('http://', 'https://')):
            url = 'http://' + url
        
        # Try to open the website (just check if reachable)
        urllib.request.urlopen(url, timeout=5)
        return 0  # reachable = normal
    except (URLError, HTTPError) as e:
        print(f"Network Error for {url}: {e}")
        return 1  # unreachable = abnormal
    except Exception as e:
        print(f"Other Error for {url}: {e}")
        return 1


In [71]:
splitted_data['web_traffic'] = raw_data['websites'].apply(web_traffic)

Network Error for http://groups.yahoo.com/group/voice_actor_appreciation/links/events_and_appearanc_000985383761/: HTTP Error 429: Too Many Requests
Network Error for http://voice-international.com/: HTTP Error 404: Not Found
Network Error for http://voicechasers.com/forum/viewforum.php?f=8: HTTP Error 503: Service Temporarily Unavailable
Network Error for http://www.geocities.com/hollywood/hills/8944/: HTTP Error 429: Too Many Requests
Other Error for http://asifa.proboards61.com/index.cgi?action=calendarviewall: Remote end closed connection without response
Network Error for http://groups.yahoo.com/group/voice_actor_appreciation/cal///group/voice_actor_appreciation/?v=42&amp;pos=: HTTP Error 429: Too Many Requests
Network Error for http://us.imdb.com/name/nm0267724/: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'us.imdb.com'. (_ssl.c:1020)>
Network Error for http://teamknightrider.com/cast/plato/plato.html:

In [72]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token,abnormal_url,google_index,web_traffic
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0,0,normal,1,0
1,http,danoday.com,summit.shtml,0,0,0,0,0,0,0,0,normal,1,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0,0,0,0,0,normal,1,1
3,http,voice-international.com,,0,0,0,1,0,0,0,0,normal,1,1
4,http,www.livinglegendsltd.com,,0,0,0,0,0,0,0,0,normal,1,0


Feature - 11

Domain Registration Length

Based on the fact that a phishing website lives for a short period of time, 
we believe that trustworthy domains are regularly paid for several years in advance. 
In our dataset, we find that the longest fraudulent domains have been used for one year only.

Rule: IF{Domains Expires on≤ 1 years → Phishing
         
         Otherwise→ Legitimate

In [73]:
import whois
from datetime import datetime
import time
def domain_registration_length_sub(domain):
    expiration_date = domain.expiration_date
    today = time.strftime('%Y-%m-%d')
    today = datetime.strptime(today, '%Y-%m-%d')
    if expiration_date is None:
        return 1
    elif type(expiration_date) is list or type(today) is list :
        return 2             #If it is a type of list then we can't select a single value from list. So,it is regarded as suspected website  
    else:
        registration_length = abs((expiration_date - today).days)
        if registration_length / 365 <= 1:
            return 1
        else:
            return 0

    
    
    

In [74]:
def domain_registration_length_main(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return domain_registration_length_sub(domain_name)
    

In [75]:
splitted_data['domain_registration_length'] = splitted_data['domain_name'].apply(domain_registration_length_main)

In [76]:
splitted_data.head()

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token,abnormal_url,google_index,web_traffic,domain_registration_length
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0,0,normal,1,0,1
1,http,danoday.com,summit.shtml,0,0,0,0,0,0,0,0,normal,1,0,1
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0,0,0,0,0,normal,1,1,2
3,http,voice-international.com,,0,0,0,1,0,0,0,0,normal,1,1,2
4,http,www.livinglegendsltd.com,,0,0,0,0,0,0,0,0,normal,1,0,2


In [77]:
#Testing the above function
domain_registration_length_main('www.google.com')

2

In [78]:
domain_registration_length_main('www.w3schools.com')

1

Feature - 12

This feature can be extracted from WHOIS database (Whois 2005). 
Most phishing websites live for a short period of time. 
By reviewing our dataset, we find that the minimum age of the legitimate domain is 6 months.

Rule: IF {Age Of Domain≥6 months → Legitimate
          
          Otherwise → Phishing

In [79]:
def age_of_domain_sub(domain):
    creation_date = domain.creation_date
    expiration_date = domain.expiration_date
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    elif ((type(expiration_date) is list) or (type(creation_date) is list)):
        return 2
    else:
        ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain/30) < 6):
            return 1
        else:
            return 0

In [80]:
def age_of_domain_main(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return age_of_domain_sub(domain_name)



In [81]:
splitted_data['age_of_domain'] = splitted_data['domain_name'].apply(age_of_domain_main)

In [82]:
splitted_data

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token,abnormal_url,google_index,web_traffic,domain_registration_length,age_of_domain
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0,0,normal,1,0,1,1
1,http,danoday.com,summit.shtml,0,0,0,0,0,0,0,0,normal,1,0,1,2
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0,0,0,0,0,normal,1,1,2,2
3,http,voice-international.com,,0,0,0,1,0,0,0,0,normal,1,1,2,2
4,http,www.livinglegendsltd.com,,0,0,0,0,0,0,0,0,normal,1,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,http,www.post-gazette.com,magazine/19990223voicetalent1.asp,2,0,0,1,0,0,0,0,normal,1,1,0,0
97,http,www.serkworks.com,roommates/index.htm,0,0,0,0,0,0,0,0,normal,1,1,1,0
98,http,www.armory.com,~keeper/jesshirt.html,0,0,0,0,0,0,0,0,normal,1,1,0,0
99,http,www.voicechasers.com,database/showactor.php?actorid=1220,2,0,0,0,0,0,0,0,normal,1,1,1,0


Feature - 13

DNS Record

For phishing websites, either the claimed identity is not recognized by the WHOIS database (Whois 2005) or 
no records founded for the hostname (Pan and Ding 2006). 
If the DNS record is empty or not found then the website is classified as “Phishing”, otherwise it is classified as “Legitimate”.

Rule: IF{no DNS Record For The Domain → Phishing
         
         Otherwise→ Legitimate
        

In [83]:
def dns_record(domain):
    dns = 0
    try:
        domain_name = whois.whois(domain)
        print(domain_name)
    except:
        dns = 1
        
    if dns == 1:
        return 1
    else:
        return dns



In [84]:
splitted_data['dns_record'] = splitted_data['domain_name'].apply(dns_record)

{
  "domain_name": null,
  "registrar": null,
  "registrar_url": null,
  "reseller": null,
  "whois_server": null,
  "referral_url": null,
  "updated_date": null,
  "creation_date": null,
  "expiration_date": null,
  "name_servers": null,
  "status": null,
  "emails": null,
  "dnssec": null,
  "name": null,
  "org": null,
  "address": null,
  "city": null,
  "state": null,
  "registrant_postal_code": null,
  "country": null
}
{
  "domain_name": "DANODAY.COM",
  "registrar": "DREAMHOST",
  "registrar_url": [
    "http://www.DreamHost.com",
    "WWW.DREAMHOST.COM"
  ],
  "reseller": null,
  "whois_server": "whois.dreamhost.com",
  "referral_url": null,
  "updated_date": "2025-02-07 09:08:36",
  "creation_date": [
    "1997-03-10 05:00:00",
    "1997-03-09 21:00:00"
  ],
  "expiration_date": "2026-03-11 04:00:00",
  "name_servers": [
    "A.DNS.HOSTWAY.NET",
    "B.DNS.HOSTWAY.NET"
  ],
  "status": [
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clien

In [85]:
splitted_data

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token,abnormal_url,google_index,web_traffic,domain_registration_length,age_of_domain,dns_record
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0,0,normal,1,0,1,1,0
1,http,danoday.com,summit.shtml,0,0,0,0,0,0,0,0,normal,1,0,1,2,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0,0,0,0,0,normal,1,1,2,2,0
3,http,voice-international.com,,0,0,0,1,0,0,0,0,normal,1,1,2,2,0
4,http,www.livinglegendsltd.com,,0,0,0,0,0,0,0,0,normal,1,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,http,www.post-gazette.com,magazine/19990223voicetalent1.asp,2,0,0,1,0,0,0,0,normal,1,1,0,0,0
97,http,www.serkworks.com,roommates/index.htm,0,0,0,0,0,0,0,0,normal,1,1,1,0,0
98,http,www.armory.com,~keeper/jesshirt.html,0,0,0,0,0,0,0,0,normal,1,1,0,0,0
99,http,www.voicechasers.com,database/showactor.php?actorid=1220,2,0,0,0,0,0,0,0,normal,1,1,1,0,0


Feature - 14

Statistical-Reports Based Feature

Several parties such as PhishTank (PhishTank Stats, 2010-2012),
and StopBadware (StopBadware, 2010-2012) formulate numerous statistical reports on phishing websites at every given period of time
some are monthly and others are quarterly. 

Rule: IF{Host Belongs to Top Phishing IPs or Top Phishing Domains → Phishing
         
         Otherwise → Legitimate


In [87]:
import socket
def statistical_report(url):
    hostname = url
    h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
    z = int(len(h))
    if z != 0:
        y = h[0][1]
        hostname = hostname[y:]
        h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
        z = int(len(h))
        if z != 0:
            hostname = hostname[:h[0][0]]
    url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
    try:
        ip_address = socket.gethostbyname(hostname)
        ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|175\.126\.123\.219|141\.8\.224\.221|10\.10\.10\.10|43\.229\.108\.32|103\.232\.215\.140|69\.172\.201\.153|216\.218\.185\.162|54\.225\.104\.146|103\.243\.24\.98|199\.59\.243\.120|31\.170\.160\.61|213\.19\.128\.77|62\.113\.226\.131|208\.100\.26\.234|195\.16\.127\.102|195\.16\.127\.157|34\.196\.13\.28|103\.224\.212\.222|172\.217\.4\.225|54\.72\.9\.51|192\.64\.147\.141|198\.200\.56\.183|23\.253\.164\.103|52\.48\.191\.26|52\.214\.197\.72|87\.98\.255\.18|209\.99\.17\.27|216\.38\.62\.18|104\.130\.124\.96|47\.89\.58\.141|78\.46\.211\.158|54\.86\.225\.156|54\.82\.156\.19|37\.157\.192\.102|204\.11\.56\.48|110\.34\.231\.42',ip_address)  
    except:
        return 1

    if url_match:
        return 1
    else:
        return 0


<>:13: SyntaxWarning: invalid escape sequence '\.'
<>:16: SyntaxWarning: invalid escape sequence '\.'
<>:13: SyntaxWarning: invalid escape sequence '\.'
<>:16: SyntaxWarning: invalid escape sequence '\.'
C:\Users\User\AppData\Local\Temp\ipykernel_1936\2380061626.py:13: SyntaxWarning: invalid escape sequence '\.'
  url_match=re.search('at\.ua|usa\.cc|baltazarpresentes\.com\.br|pe\.hu|esy\.es|hol\.es|sweddy\.com|myjino\.ru|96\.lt|ow\.ly',url)
C:\Users\User\AppData\Local\Temp\ipykernel_1936\2380061626.py:16: SyntaxWarning: invalid escape sequence '\.'
  ip_match=re.search('146\.112\.61\.108|213\.174\.157\.151|121\.50\.168\.88|192\.185\.217\.116|78\.46\.211\.158|181\.174\.165\.13|46\.242\.145\.103|121\.50\.168\.40|83\.125\.22\.219|46\.242\.145\.98|107\.151\.148\.44|107\.151\.148\.107|64\.70\.19\.203|199\.184\.144\.27|107\.151\.148\.108|107\.151\.148\.109|119\.28\.52\.61|54\.83\.43\.69|52\.69\.166\.231|216\.58\.192\.225|118\.184\.25\.86|67\.208\.74\.71|23\.253\.126\.58|104\.239\.157\.210|17

In [88]:
splitted_data['statistical_report'] = raw_data['websites'].apply(statistical_report)

In [89]:
splitted_data

,protocol,domain_name,address,long_url,having_@_symbol,redirection_//_symbol,prefix_suffix_seperation,sub_domains,having_ip_address,shortening_service,https_token,abnormal_url,google_index,web_traffic,domain_registration_length,age_of_domain,dns_record,statistical_report
0,http,www.emuck.com:3000,archive/egan.html,0,0,0,0,0,0,0,0,normal,1,0,1,1,0,1
1,http,danoday.com,summit.shtml,0,0,0,0,0,0,0,0,normal,1,0,1,2,0,0
2,http,groups.yahoo.com,group/voice_actor_appreciation/links/events_an...,1,0,0,0,0,0,0,0,normal,1,1,2,2,0,0
3,http,voice-international.com,,0,0,0,1,0,0,0,0,normal,1,1,2,2,0,0
4,http,www.livinglegendsltd.com,,0,0,0,0,0,0,0,0,normal,1,0,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,http,www.post-gazette.com,magazine/19990223voicetalent1.asp,2,0,0,1,0,0,0,0,normal,1,1,0,0,0,0
97,http,www.serkworks.com,roommates/index.htm,0,0,0,0,0,0,0,0,normal,1,1,1,0,0,0
98,http,www.armory.com,~keeper/jesshirt.html,0,0,0,0,0,0,0,0,normal,1,1,0,0,0,0
99,http,www.voicechasers.com,database/showactor.php?actorid=1220,2,0,0,0,0,0,0,0,normal,1,1,1,0,0,0


In [90]:
domain_info = whois.whois('example.com')
print(domain_info)

{
  "domain_name": "EXAMPLE.COM",
  "registrar": "RESERVED-Internet Assigned Numbers Authority",
  "registrar_url": "http://res-dom.iana.org",
  "reseller": null,
  "whois_server": "whois.iana.org",
  "referral_url": null,
  "updated_date": "2024-08-14 07:01:34",
  "creation_date": "1995-08-14 04:00:00",
  "expiration_date": "2025-08-13 04:00:00",
  "name_servers": [
    "A.IANA-SERVERS.NET",
    "B.IANA-SERVERS.NET"
  ],
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited",
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited"
  ],
  "emails": null,
  "dnssec": "signedDelegation",
  "name": null,
  "org": null,
  "address": null,
  "city": null,
  "state": null,
  "registrant_postal_code": null,
  "country": null
}
